In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
train_test_df = pd.read_csv("./B02.csv")
train_test_df.info()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260115 entries, 0 to 260114
Data columns (total 17 columns):
id                                              260115 non-null object
teacher_id                                      260115 non-null object
teacher_prefix                                  260110 non-null object
school_state                                    260115 non-null object
project_submitted_datetime                      260115 non-null object
project_grade_category                          260115 non-null object
project_subject_categories                      260115 non-null object
project_subject_subcategories                   260115 non-null object
project_title                                   260115 non-null object
project_essay_1                                 260115 non-null object
project_essay_2                                 260115 non-null object
project_essay_3                                 9078 non-null object
project_essay_4                        

In [3]:
train_test_df['school_state'].unique()

array(['NV', 'GA', 'UT', 'NC', 'CA', 'DE', 'MO', 'SC', 'IN', 'IL', 'VA',
       'PA', 'NY', 'FL', 'NJ', 'TX', 'LA', 'ID', 'OH', 'OR', 'MD', 'WA',
       'MA', 'KY', 'AZ', 'MI', 'CT', 'AR', 'WV', 'NM', 'WI', 'MN', 'OK',
       'AL', 'TN', 'IA', 'KS', 'CO', 'DC', 'WY', 'NH', 'HI', 'SD', 'MT',
       'MS', 'RI', 'VT', 'ME', 'NE', 'AK', 'ND'], dtype=object)

In [4]:
hot_list = ['NV', 'UT', 'CA', 'IL', 'OK', 'AZ', 'MI', 'CT', 'AR', 'WV']
train_test_df['project_is_approved'] = train_test_df['school_state'].apply(lambda x: 1 if x in hot_list else 0)

In [5]:
train_test_df[train_test_df['project_is_approved']==1].head(100)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,total_price
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1,899.94
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1,469.99
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1,711.00
9,p191410,2071fb0af994f8f16e7c6ed0f35062a1,Mrs.,IL,2016-09-24 18:38:59,Grades PreK-2,Literacy & Language,Literacy,Snuggle Up With A Good Book,I teach first grade in a small farming town in...,There is nothing better than snuggling up with...,NaN,NaN,My students need 2 youth sized reclining chair...,12,1,119.76
11,p226941,103cc1667cf9361bf1c58c8425e76e95,Mrs.,CA,2016-09-05 19:28:32,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Technology Boost!,My children come to school everyday with the s...,"With our new iPads, my students will be able t...",NaN,NaN,My students need a projector and tablets to al...,1,1,733.95
27,p107356,f4594111d55f22ee3ed5e8e4a1c2852b,Ms.,UT,2016-10-11 19:16:18,Grades 3-5,"Math & Science, Literacy & Language","Environmental Science, Literature & Writing",Can you hear me now?,Our classroom is filled with bright smiles and...,Chromebooks computers are used daily in our cl...,NaN,NaN,My students need earbud headphones to use in t...,6,1,125.10
34,p139731,d89c560612543869db2ca395d5831759,Mr.,CA,2017-01-10 01:24:00,Grades PreK-2,"Health & Sports, Literacy & Language","Health & Wellness, Literature & Writing",Happy Healthy Kinderlandia,"ICS students are bilingual, biliterate, cultur...",Our mission for the 2016-2017 school-year is t...,NaN,NaN,My students need playground equipment to be ha...,2,1,655.61
39,p047586,e35a28822e3417778ce9721261175e6f,Mrs.,IL,2016-12-05 20:14:48,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Learning Can Be Fun,Some students have autism and others have Down...,My students are hands on learners and most of ...,NaN,NaN,My students need launch and learn alphabet gam...,57,1,179.95
40,p006436,0c1e77b114858b41fd52aba3610ecb55,Ms.,IL,2017-01-10 10:30:49,Grades 9-12,"Health & Sports, Special Needs","Health & Wellness, Special Needs",Diverse Learners Need Chairs That Aren't Taped...,"\""Teach in the way a child learns, don't force...",The Diverse Learners in my classroom currently...,NaN,NaN,My students need chairs to sit on while at the...,6,1,579.66
42,p220000,87d06a109a06e632ddc1a94467cae00f,Ms.,IL,2017-03-01 20:52:05,Grades PreK-2,Math & Science,"Applied Sciences, Mathematics",Let's Let Out Some S.T.E.A.M.,Can you remember the day you started kindergar...,"STEAM (Science, Technology, Engineering, Art, ...",NaN,NaN,My students need an opportunity to be great wi...,1,1,499.94


In [6]:
train_test_df[train_test_df['project_is_approved']==0].head(100)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,total_price
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,400.00
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0,684.47
5,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,0,727.36
6,p103285,4e156c5fb3eea2531601c8736f3751a7,Mrs.,MO,2016-08-31 00:30:43,Grades PreK-2,Health & Sports,Health & Wellness,Active Kindergartners,Kindergarten is the new first grade. My studen...,With balance discs and stools as flexible seat...,NaN,NaN,My students need stability stools and inflatab...,1,0,414.02
7,p181781,c71f2ef13b4bc91afac61ca8fd4c0f9f,Mrs.,SC,2016-08-03 13:26:01,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literature & Writing",Fabulous Firsties-Wiggling to Learn!,First graders are fantastic! They are excited ...,First graders love learning! We need 6 wiggle-...,NaN,NaN,My students need wiggle stools to allow them t...,0,0,414.78
8,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,Math & Science,Mathematics,Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,0,319.80
10,p030093,b9e731e16ad8669f37a43e5316518106,Teacher,VA,2017-03-09 17:16:13,Grades PreK-2,"Literacy & Language, Math & Science","Literature & Writing, Mathematics","Writing, Writing, Writing!",My classroom has 24 students in it. We have an...,"In our classroom, we use dry erase boards thro...",NaN,NaN,My students need individual dry erase boards t...,1,0,102.00
12,p225747,4681d60136ea859084ab488f2254b219,Ms.,PA,2016-12-08 13:25:03,Grades 3-5,"Applied Learning, Literacy & Language","Character Education, Literacy",Life Lessons through Literacy,"My school is located in a high poverty area, j...",As a Reading Specialist who serves Grades 1-8 ...,NaN,NaN,My students need books that teach life lessons...,3,0,176.59
13,p173555,9b7f355e34bc9ca5740779b69ee14d8e,Mrs.,NY,2016-11-15 22:13:39,Grades 3-5,Literacy & Language,Literature & Writing,Extra! Extra! Read all about it!! We love to ...,"Each day my fifth graders walk into our \""home...",My students have had a taste of good reading! ...,NaN,NaN,"My students need good books, with life lessons...",5,0,871.27
14,p055350,882c8ddea1d5c4c31976e34f20b461f9,Ms.,FL,2017-02-02 13:46:56,Grades PreK-2,Special Needs,Special Needs,Flexible seating,The children at our school come from a variety...,"After fifteen years of teaching, we have seen ...",NaN,NaN,My students need flexible seating options to p...,0,0,374.38


In [7]:
train_test_df.isnull().sum()

id                                                   0
teacher_id                                           0
teacher_prefix                                       5
school_state                                         0
project_submitted_datetime                           0
project_grade_category                               0
project_subject_categories                           0
project_subject_subcategories                        0
project_title                                        0
project_essay_1                                      0
project_essay_2                                      0
project_essay_3                                 251037
project_essay_4                                 251037
project_resource_summary                             0
teacher_number_of_previously_posted_projects         0
project_is_approved                                  0
total_price                                          0
dtype: int64

In [8]:
train_corr = train_test_df.corr()
train_corr

,teacher_number_of_previously_posted_projects,project_is_approved,total_price
teacher_number_of_previously_posted_projects,1.000000,0.032622,-0.034801
project_is_approved,0.032622,1.000000,0.015143
total_price,-0.034801,0.015143,1.000000


In [9]:
from my_modules.my_encoder import CategoryValueEncoder as CVE
from my_modules.my_encoder import TextValueEncoder as TVE
from my_modules.my_encoder import DateValueEncoder as DVE
from my_modules.my_encoder import NumericValueEncoder as NVE

from sklearn.feature_extraction.text import CountVectorizer
project_title = train_test_df['project_title']
#enc1 = TVE(df1[0:30000])
enc1 = TVE(project_title)
# 出現頻度が1%未満のデータは無視する。
# 理由：1回でた単語のデータのラベルが正であっても負であっても、その単語との因果関係を認められないから。
# We get ValueError when get any data. You should enhance range. 
enc1.vectorizer = CountVectorizer(min_df=0.01, max_df=1.00, stop_words="english") 
project_title_endoded = enc1.to_bow_encoding()
#names = enc1.vectorizer.get_feature_names()

encode_df = pd.DataFrame()
encode_df = pd.concat([encode_df,
                            train_test_df['id'],                             
                            train_test_df['project_is_approved'],
                            CVE(train_test_df['project_grade_category']).to_binary_encoding(),
                            CVE(train_test_df['school_state']).to_binary_encoding(),
                            CVE(train_test_df['teacher_prefix']).to_binary_encoding(),
                            CVE(DVE(train_test_df['project_submitted_datetime']).to_year()).to_binary_encoding(),
                            CVE(DVE(train_test_df['project_submitted_datetime']).to_month()).to_binary_encoding(),
                            CVE(DVE(train_test_df['project_submitted_datetime']).to_day()).to_binary_encoding(),
                            CVE(DVE(train_test_df['project_submitted_datetime']).to_dayofweek()).to_binary_encoding(),
                            TVE(train_test_df['project_subject_categories']).to_bow_encoding(),
                            TVE(train_test_df['project_subject_subcategories']).to_bow_encoding(),  
                            #TVE(train_test_df['project_title']).to_bow_encoding(),  
                            project_title_endoded,
                            NVE(train_test_df['teacher_number_of_previously_posted_projects']).normalize(),
                            NVE(train_test_df['total_price']).normalize()                          
                           ], axis=1) 

In [10]:
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260115 entries, 0 to 260114
Columns: 148 entries, id to total_price
dtypes: float64(2), int64(145), object(1)
memory usage: 293.7+ MB


In [11]:
encode_df.head(5)

,id,project_is_approved,project_grade_category_enc_0,project_grade_category_enc_1,project_grade_category_enc_2,school_state_enc_0,school_state_enc_1,school_state_enc_2,school_state_enc_3,school_state_enc_4,...,project_title_enc_44,project_title_enc_45,project_title_enc_46,project_title_enc_47,project_title_enc_48,project_title_enc_49,project_title_enc_50,project_title_enc_51,teacher_number_of_previously_posted_projects,total_price
0,p036502,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.55,0.56
1,p039565,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.46,0.47
2,p233823,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.48,0.49
3,p185307,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.52,0.53
4,p013780,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.61,0.53


In [12]:
encode_df[182070:182090]

,id,project_is_approved,project_grade_category_enc_0,project_grade_category_enc_1,project_grade_category_enc_2,school_state_enc_0,school_state_enc_1,school_state_enc_2,school_state_enc_3,school_state_enc_4,...,project_title_enc_44,project_title_enc_45,project_title_enc_46,project_title_enc_47,project_title_enc_48,project_title_enc_49,project_title_enc_50,project_title_enc_51,teacher_number_of_previously_posted_projects,total_price
182070,p068185,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0.46,0.48
182071,p248714,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0.46,0.46
182072,p045565,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0.46,0.50
182073,p078709,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0.46,0.62
182074,p184627,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0.69,0.46
182075,p014188,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0.46,0.45
182076,p116452,1,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0.46,0.45
182077,p074761,1,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0.63,0.53
182078,p136737,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0.56,0.44
182079,p190772,0,0,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0.46,0.46


In [13]:
train_df = encode_df[:182080]

In [14]:
train_df.to_csv("C01_train_encoded.csv", index=False)

In [15]:
test_df = encode_df[182080:]

In [16]:
test_df.to_csv("C01_test_encoded.csv", index=False)